In [1]:
import os
import tempfile

import torch
from dotenv import load_dotenv
from IPython.display import Audio, display
from melo.api import TTS
from path import Path

from openvoice import se_extractor
from openvoice.api import ToneColorConverter

/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
load_dotenv()
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = Path(tempfile.gettempdir()) / 'outputs_v2'
print(device)

cuda:0


In [3]:
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')
os.makedirs(output_dir, exist_ok=True)

/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializ

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [4]:
reference_speaker = os.getenv('REFERENCE_SPEAKER')
if not reference_speaker:
    raise ValueError('Specify reference speaker in .env file')
target_se, _ = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)
src_path = output_dir / 'tmp.wav'

OpenVoice version: v2


[2024-12-03 16:54:11.631] [ctranslate2] [thread 10064] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.
/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/torch/functional.py:704: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]


In [30]:
language = 'JP'  # EN_NEWEST, JP
model = TTS(language=language, device=device)
if len(model.hps.data.spk2id) > 1:
    raise ValueError(f'There are several speaker ids: {model.hps.data.spk2id}')

/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/mnt/ubuntu_storage/projects/OpenVoice/.venv/lib/python3.9/site-packages/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seria

In [44]:
text = "じんるい"
speaker_key = language.lower().replace('_', '-')

source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
model.tts_to_file(
    text,
    speaker_id=0,
    output_path=src_path,
    speed=1,
    sdp_ratio=0.5,
    # noise_scale=0.3,
    # noise_scale_w=0.5,
    quiet=True,
)
save_path = output_dir / 'audio_with_cloned_voice.wav'

audio = Audio(src_path)
display(audio)

tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    tau=0.3,
)
audio = Audio(save_path)
display(audio)

/tmp/ipykernel_10064/2763129047.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_locatio